# Using Chatgpt based on Langchain

## Models

In [2]:
import os
import openai
from dotenv import load_dotenv,find_dotenv

In [4]:
_ = load_dotenv(find_dotenv())

In [6]:
# 获取环境变量openai_api_key
openai_api_key = os.environ['OPENAI_API_KEY']
# 包含API服务器基本地址的URL，制定了你用来发送请求的API服务器地址，当建立一个API
# 客户端的时候，所有的请求都会send到这个url
base_url = os.environ['BASE_URL']
# 总之
# api_key决定是否允许这个请求以及以什么级别进行访问
# base_url则决定了请求发送到哪个服务器

In [8]:
from langchain_openai import ChatOpenAI

In [10]:
llm = ChatOpenAI(temperature=0.0,openai_api_key=openai_api_key,
                base_url=base_url)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001D6C6BEF130>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D6C6C1CCA0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api.chatanywhere.tech/v1', openai_proxy='')

In [13]:
output = llm.invoke("do you know what means large-language-model")
type(output.content)

str

## prompt
开发大模型应用时候，通常不会直接将用户的输入直接传递给LLM。
需要用到提示模板：它捆绑了从用户输入转换成完全格式化的提示的所有逻辑

In [50]:
# learning

In [52]:
from langchain.prompts import PromptTemplate
# 定义一个提示模板
template  = PromptTemplate(
    input_variables = ["name","question"],
    template = "hi,{name}.the question your asked is: {question}, the answer I provided is as followed:"
)
# 定义模板中所需要填充的变量
variables = {
    "name":"Brian",
    "question":"how to use langchain"
}
# 渲染模板，生成最终得提示
final_prompt = template.format(**variables)
final_prompt

'hi,Brian.the question your asked is: how to use langchain, the answer I provided is as followed:'

In [54]:
from langchain_core.prompts import ChatPromptTemplate
# we ask the model to translate the texts we give to chinese
prompt = """请你将由三个反引号分割的文本翻译成英文！\
text:'''{text}'''
"""

In [55]:
text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
# 构造完整的提示词模板
prompt.format(text=text)

"请你将由三个反引号分割的文本翻译成英文！text:'''我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。'''\n"

In [57]:
# 通常而言，一个ChatMessageTemplate 是一个ChatMessageTemplate的列表。
# 每个ChatMessageTemplate包含格式化聊天消息的说明（角色以及其内容）
from langchain.prompts.chat import ChatPromptTemplate
template = "you are an agent, you can help me to translate {input_lan} into {output_lan}"
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

messages = chat_prompt.format_messages(input_lan = "Chinese", output_lan = "English",text=text)
messages

[SystemMessage(content='you are an agent, you can help me to translate Chinese into English'),
 HumanMessage(content='我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。')]

In [58]:
output = llm.invoke(messages)
output

AIMessage(content='I carried luggage heavier than my body and waded into the bottom of the Nile. After a few flashes of lightning, I saw a pile of halos, not sure if this is the place.', response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 78, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'stop', 'logprobs': None}, id='run-84c28910-02c2-4dab-a3de-0d4004577248-0', usage_metadata={'input_tokens': 78, 'output_tokens': 40, 'total_tokens': 118})

## 输出解析器

In [61]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output = output_parser.invoke(output)
output

'I carried luggage heavier than my body and waded into the bottom of the Nile. After a few flashes of lightning, I saw a pile of halos, not sure if this is the place.'

## complete process

将上述所有的步骤组合成一条链，该链：
1.获取输入变量
2.将这些变量传递给提示模板以创建提示，将提示传递给语言模型
3.通过输出解析器传递输出

In [62]:
chain = chat_prompt | llm | output_parser
chain.invoke({"input_lan":"Chinese","output_lan":"English","text":text})

'I carried luggage heavier than my body and waded into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, unsure if this is the place.'

In [63]:
text = 'I carried luggage heavier than my body and waded into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, unsure if this is the place.'
chain.invoke({"input_lan":"English","output_lan":"Chinese","text":text})

'我扛着比我的身体还要重的行李，涉入了尼罗河的深处。穿过几道闪电后，我看到了一堆光环，不确定这是否就是目的地。'

# Using wenxin(QIANFAN)

In [64]:
from dotenv import find_dotenv, load_dotenv
import os

In [66]:
_ = load_dotenv(find_dotenv())

In [71]:
QIANFAN_AK = os.environ["QIANFAN_AK"]
QIANFAN_SK = os.environ["QIANFAN_SK"]
print(QIANFAN_AK,QIANFAN_SK,sep="\n------------\n")

3bXSuko4LWd4QW2Bdf8tQsWA
------------
vtNKZdQYs5xObTbQCGESdAm1CYyVyxYv


In [73]:
%pip install -qU langchain langchain-community

Note: you may need to restart the kernel to use updated packages.


In [77]:
from langchain_community.llms import QianfanLLMEndpoint

In [82]:
llm = QianfanLLMEndpoint(streaming = True)
res = llm("really nice to see you")
res

"I'm glad to see you too. Thank you for your positive feedback. I will continue to work hard to provide you with better services. If you have any suggestions or problems, please feel free to tell me. I will do my best to solve them."

# Using SPARK

In [110]:
_ = load_dotenv(find_dotenv())
IFLYTEK_SPARK_APP_ID = os.environ["IFLYTEK_SPARK_APP_ID"]
IFLYTEK_SPARK_API_KEY = os.environ["IFLYTEK_SPARK_API_KEY"]
IFLYTEK_SPARK_API_SECRET = os.environ["IFLYTEK_SPARK_API_SECRET"]

In [111]:
def gen_spark_params(model):
    """
    构造星火模型请求参数
    """
    spark_url_tpl = "wss://spark-api.xf-yun.com/{}/chat"
    model_params_dict = {
        "v1.5": {
            "domain": "general", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v1.1") # 云端环境的服务地址
        },
        # v2.0 版本
        "v2.0": {
            "domain": "generalv2", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v2.1") # 云端环境的服务地址
        },
        # v3.0 版本
        "v3.0": {
            "domain": "generalv3", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v3.1") # 云端环境的服务地址
        },
        # v3.5 版本
        "v3.5": {
            "domain": "generalv3.5", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v3.5") # 云端环境的服务地址
        }
    }
    return model_params_dict[model]

In [113]:
from langchain_community.llms import SparkLLM
spark_api_url = gen_spark_params(model="v3.5")["spark_url"]
llm = SparkLLM(spark_api_url = spark_api_url)

In [114]:
res = llm("hi,please introduce yourself")
res

'Hello! I am iFLYTEK Spark, and I can provide you with various cognitive intelligence services including language understanding, question and answer, reasoning and so on through natural language interaction.'

# Using ChatGLM3

In [129]:
from zhipuai_llm import ZhipuAILLM

In [132]:
api_key = os.environ["ZHIPUAI_API_KEY"]
api_key

'32b0876eda94b85d1d0081eafb5aad2e.1tUeCGG9ElLcJiAO'

In [147]:
zhipuai_model = ZhipuAILLM(model = 'glm-4',temperature=0, api_key=api_key)

In [151]:
zhipuai_model("请介绍一下智谱AI的张鹏")

'智谱AI的张鹏是该公司的首席执行官。根据提供的参考信息，张鹏在人工智能领域具有深刻的见解和丰富的经验。他强调人工智能的发展对于各行各业的重要性，并认为大模型技术将在其中扮演核心角色。\n\n张鹏关注于推动大模型的终端侧应用研发，并致力于提升人机交互能力。他认为，随着AI技术向终端下沉，智能化的体验将变得更加触手可及。在他的领导下，智谱AI专注于打造新一代认知智能大模型，并实现了大模型生成式AI的全链路自主可控。\n\n张鹏还曾提到，智谱AI研发的新一代基座大模型GLM-4在中文能力上可以比肩GPT-4，展示了公司在技术上的实力和进步。此外，他提倡开源生态的繁荣，并计划为大模型开源基金提供支持。\n\n在人工智能的发展阶段上，张鹏将其分为三个阶段：符号AI、感知智能和认知智能。他认为当前正处于认知智能阶段，关注的是认知的可计算性。张鹏还提出了“2024年是AGI元年”的观点，意味着人工智能正进入一个全新的通用人工智能时代。\n\n作为公司的领导者，张鹏强调智谱AI将继续与终端侧产业伙伴合作，开拓新的商用AI终端应用，为各行各业带来数字化和智能化的先进工具。通过他的领导和远见，智谱AI在人工智能领域不断探索和创新。'